In [ ]:
import pyodbc
import pandas as pd
from datetime import datetime

# Configurar conexão com o SQL Server
conn_str = (
    r'DRIVER={SQL Server};'
    r'SERVER=DESKTOP-5751I66;'
    r'DATABASE=VendasCursosOnline;'
    r'Trusted Connection=yes;'  
)

conn = pyodbc.connect(conn_str)
cursor = conn.cursor()


# Carregar dados do Excel
df = pd.read_excel(r'I:\ESTUDO\PROJETOS\Python\Projet.2_E-commerce\Avaliação de Vendas de Cursos Onlines.xlsx',sheet_name='Base Tratada')


# Função para validar a coluna de datas
def validate_dates(data_column):
        # Verificar o tipo de dados
        dtype = data_column.dtype
        print(f"Tipo de dados da coluna Data: {dtype}")
        print(f"Primeiros valores da coluna Data\n{data_column.head()}")
        
        
        if dtype == 'datetime64[ns]':
            print("A coluna 'Data'já está no formato datetime64[ns]. Nunhuma conversão é necessária.")
            return True
        
        # Verificar se há valores NaN ou fora do intervalo esperado
        if dtype in['float64', 'int64']:
            if data_column.isna().any():
                print("Erro: A coluna 'Data' contém valores NaN.")
                return False
        
        # Verificar intervalo plausível (exemplo: 0 a 100000 dias a partir de 1899-12-30)
            min_days, max_days =0, 100000
            if (data_column < min_days).any() or (data_column > max_days).any():
                print(f"Erro: A coluna 'Data'contém valores fora do intervalo esperado({min_days} a {max_days}) dias.)")
                return False
        
            # Verificar se todos os valores são numéricos inteiros ou floats válidos
            if not data_column.apply(lambda x: isinstance(x,(int, float))).all():
                print("Erro: A coluna 'Data'Contém valores que não são numéricos.")
                return False
        
            print("A coluna 'Data' está no formato correto para conversão.")
            return True
    
        print(f"Erro: A coluna 'Data' tem tipo inesperado({dtype}).Esperado: float64, int64 ou datetime64[ns].")    
            
         
"""Valida se a coluna de datas está no formato correto para conversão.
    Retorna True se válida, False caso contrário, com mensagens de erro."""


# Limpeza de dados
if validate_dates(df['Data']):
    if df['Data'].dtype != 'datetime64[ns]':
        # Converter a coluna 'Data' para datetime
        try:
            df['Data'] = pd.to_datetieme(df['Data'], origin='1899-12-30')
            print("Conversão de datas realizada com sucesso.")
            
        except Exception as e:
            print(f"Erro ao converter a coluna 'Data': {e}")   
            raise
else:
    raise ValueError("Validação da coluna 'Data' falhou.Verifique os dados e tente novamente.")

# Remover colunas desnecessárias
df = df.drop_duplicates()
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')
df = df.fillna('')


# Inserir dados no SQL Server
for index, row in df.iterrows():
    cursor.execute('''
                   INSERT INTO Vendas (
                                Data, Ano, Mes,Vendedor, 
                                Cliente,Regiao, Produto, Valor
                        )
                         VALUES  (?, ?, ?, ?, ?, ?, ?, ?)
                   ''', (
                       row['Data'].strftime('%Y-%m-%d'),
                       row['Ano'],
                       row['Mes'],
                       row['Vendedor'],
                       row['Cliente'],
                       row['Regiao'],
                       row['Produto'],
                       row['Valor']
                         ))
conn.commit()
cursor.close()
conn.close()
print("Dados inseridos com sucesso no SQL Server.")

Tipo de dados da coluna Data: datetime64[ns]
Primeiros valores da coluna Data
0   2019-01-01
1   2019-01-01
2   2019-01-02
3   2019-01-02
4   2019-01-02
Name: Data, dtype: datetime64[ns]
A coluna 'Data'já está no formato datetime64[ns]. Nunhuma conversão é necessária.
